In [1]:
import os
import pandas as pd
import time
import copy
import numpy as np


In [2]:
os.chdir(r'E:\文件\大学课程\大四上\毕业设计\数据收集\收集整理\后期数据清洗整理\LAMP2')  #设置工作路径
df=pd.read_csv(r'result.csv')
df.dropna(subset=['MIC'], inplace=True)
df

,id,name,source,mass,length,Isoelectric point,activity,sequence,MIC
0,L01A000001,CTXD_CERCA,Ceratitis capitata,3511.4 Da,36 aa,11.5,Antibacterial,SIGTAVKKAVPIAKKVGKVAIPIAKAVLSVVGQLVG,No MICs found on LAMP database
1,L01A000002,D104A_HUMAN,Homo sapiens,5712.5 Da,48 aa,9.45,Antibacterial| Antifungal,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,1 Target: Staphylococcus carnosus TM300 MIC...
2,L01A000003,latarcin 4a| pLtc 4a,Lachesana tarabaevi (Spider),2903.5 Da,24 aa,10.72,Antibacterial| Antifungal,GLKDKFKSMGEKLKQYIQTWKAKF,1 Target: A. globiformis MIC: 0.87 ug/ml ...
3,L01A000004,bombinin-like peptide 7| BPL-7,Bombina orientalis,2551.9 Da,27 aa,10.51,Antibacterial,GIGGALLSAGKSALKGLAKGLAEHFAN,No MICs found on LAMP database
4,L01A000005,bactenecin 5 - bovine,Bos taurus (cattle),5150.2 Da,42 aa,13.1,Antibacterial,RFRPPIRRPPIRPPFYPPFRPPIRPPIFPPIRPPFRPPLRFP,1 Target: Salmonella typhimurium MIC: 10.3...
...,...,...,...,...,...,...,...,...,...
12620,L11A013568,Bactenecin-7 (1-16)[P7K],NaN,2107.6 Da,16 aa,13.35,Gram-,RRIRPRKPRLPRPRPR,No MICs found on LAMP database
12621,L11A013569,Bactenecin-7 (1-16)[P7R],NaN,2135.6 Da,16 aa,13.4,Gram-,RRIRPRRPRLPRPRPR,No MICs found on LAMP database
12622,L11A013570,Bactenecin-7 (1-16)[R16K],NaN,2048.5 Da,16 aa,13.28,Gram-,RRIRPRPPRLPRPRPK,No MICs found on LAMP database
12623,L11A013571,Oncocin [D2P],NaN,2372.8 Da,19 aa,12.34,Gram+Gram-,VPKPPYLPRPRPPRRIYNR,No MICs found on LAMP database


In [3]:
#定义一个将混合多种生物targets活性数据拆分成单个target活性数据的函数
def single_target(targets):
    targets = [i for i in targets.split('     ')]
    return targets

#定义一个删除unit后面分割不干净的函数
def Unit(unit):
    unit=unit.replace('.','')
    unit=unit.replace(')','')
    unit=unit.replace(';','')
    unit=unit.replace('；','')
    return unit

#定义一个同时获得两种单位的拆分target/measure/value/unit的函数
def cut_mic(two_mics):
    j = two_mics.split(':')
    target_measure = j[1].strip()
    measure = target_measure.split(' ')[-1]
    target = target_measure.strip(measure)
    target = target.strip()
    MICs = j[2].strip().split(' ')
    if len(MICs) ==5:
        num1 = MICs[0].strip()
        unit1 = MICs[1].strip()
        num2=MICs[3].strip('(')
        unit2=MICs[4].strip(')')
        lists=[[target,measure,num1,unit1],[target,measure,num2,unit2]]
        return lists
    else:
        return [target,measure,MICs]

In [4]:
# 新建一个数据表，每次写入两行
newdf=pd.DataFrame(columns=df.columns)
newdf[['target','measure','value','unit']]=''
new_index=0
df_index=df.index    #建立一个循环，要循环完原始数据的每一行
for index in df.index:
    old_row=copy.deepcopy(list(df.loc[index]))   #一定要把旧行设置成为列表，不然输出的就是series对象
    targets=copy.deepcopy(df.loc[index,'MIC'])
    new_targets=single_target(targets)
    if new_targets!=None:
        for two_mic in single_target(targets):
            if '(' in two_mic:
                if len(cut_mic(two_mic))==2:
                    row1=old_row+cut_mic(two_mic)[0]
                    newdf.loc[new_index]=row1
                    new_index+=1
                    row2=old_row+cut_mic(two_mic)[1]
                    newdf.loc[new_index]=row2
                    new_index+=1
                # else:
                #     print(cut_mic(two_mic))

In [5]:
newdf.to_excel(r'1_CAMP.xlsx',index=False)
newdf

,id,name,source,mass,length,Isoelectric point,activity,sequence,MIC,target,measure,value,unit
0,L01A000002,D104A_HUMAN,Homo sapiens,5712.5 Da,48 aa,9.45,Antibacterial| Antifungal,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,1 Target: Staphylococcus carnosus TM300 MIC...,Staphylococcus carnosus TM300,MIC,4.5,ug/ml
1,L01A000002,D104A_HUMAN,Homo sapiens,5712.5 Da,48 aa,9.45,Antibacterial| Antifungal,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,1 Target: Staphylococcus carnosus TM300 MIC...,Staphylococcus carnosus TM300,MIC,0.787746,uM
2,L01A000002,D104A_HUMAN,Homo sapiens,5712.5 Da,48 aa,9.45,Antibacterial| Antifungal,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,1 Target: Staphylococcus carnosus TM300 MIC...,Escherichia coli BL21,MIC,100,ug/ml
3,L01A000002,D104A_HUMAN,Homo sapiens,5712.5 Da,48 aa,9.45,Antibacterial| Antifungal,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,1 Target: Staphylococcus carnosus TM300 MIC...,Escherichia coli BL21,MIC,17.5055,uM
4,L01A000002,D104A_HUMAN,Homo sapiens,5712.5 Da,48 aa,9.45,Antibacterial| Antifungal,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,1 Target: Staphylococcus carnosus TM300 MIC...,Saccharomyces cerevisiae ATCC9763,MIC,100,ug/ml
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8361,L10Q2PGA80,Temporin-1TGa,Rana tagoi,1383.7 Da,13 aa,9.69,Antimicrobial,FLPIIGKLLSGIL,1 Target: E. coli MIC: 138.37 ug/ml (100 ...,E. coli,MIC,100,uM
8362,L10Q2PGA80,Temporin-1TGa,Rana tagoi,1383.7 Da,13 aa,9.69,Antimicrobial,FLPIIGKLLSGIL,1 Target: E. coli MIC: 138.37 ug/ml (100 ...,S. aureus,MIC,20.7555,ug/ml
8363,L10Q2PGA80,Temporin-1TGa,Rana tagoi,1383.7 Da,13 aa,9.69,Antimicrobial,FLPIIGKLLSGIL,1 Target: E. coli MIC: 138.37 ug/ml (100 ...,S. aureus,MIC,15,uM
8364,L10Q800R30,PBN1,Phyllomedusa bicolor,2043.5 Da,20 aa,9.69,Antimicrobial,FLSLIPHIVSGVAALAKHLG,1 Target: S. aureus MIC: 25.5438 ug/ml (1...,S. aureus,MIC,25.5438,ug/ml


In [6]:
# 删除value列中存在空字符或者空白记录的字段，并把value列改为数字格式
new=copy.deepcopy(newdf)
new['value'] = pd.to_numeric(new['value'], errors='coerce')
new.dropna(subset=['value'], inplace=True)
new.to_excel(r'2_LAMP删改value.xlsx',index=False)
new

,id,name,source,mass,length,Isoelectric point,activity,sequence,MIC,target,measure,value,unit
0,L01A000002,D104A_HUMAN,Homo sapiens,5712.5 Da,48 aa,9.45,Antibacterial| Antifungal,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,1 Target: Staphylococcus carnosus TM300 MIC...,Staphylococcus carnosus TM300,MIC,4.500000,ug/ml
1,L01A000002,D104A_HUMAN,Homo sapiens,5712.5 Da,48 aa,9.45,Antibacterial| Antifungal,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,1 Target: Staphylococcus carnosus TM300 MIC...,Staphylococcus carnosus TM300,MIC,0.787746,uM
2,L01A000002,D104A_HUMAN,Homo sapiens,5712.5 Da,48 aa,9.45,Antibacterial| Antifungal,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,1 Target: Staphylococcus carnosus TM300 MIC...,Escherichia coli BL21,MIC,100.000000,ug/ml
3,L01A000002,D104A_HUMAN,Homo sapiens,5712.5 Da,48 aa,9.45,Antibacterial| Antifungal,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,1 Target: Staphylococcus carnosus TM300 MIC...,Escherichia coli BL21,MIC,17.505500,uM
4,L01A000002,D104A_HUMAN,Homo sapiens,5712.5 Da,48 aa,9.45,Antibacterial| Antifungal,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,1 Target: Staphylococcus carnosus TM300 MIC...,Saccharomyces cerevisiae ATCC9763,MIC,100.000000,ug/ml
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8361,L10Q2PGA80,Temporin-1TGa,Rana tagoi,1383.7 Da,13 aa,9.69,Antimicrobial,FLPIIGKLLSGIL,1 Target: E. coli MIC: 138.37 ug/ml (100 ...,E. coli,MIC,100.000000,uM
8362,L10Q2PGA80,Temporin-1TGa,Rana tagoi,1383.7 Da,13 aa,9.69,Antimicrobial,FLPIIGKLLSGIL,1 Target: E. coli MIC: 138.37 ug/ml (100 ...,S. aureus,MIC,20.755500,ug/ml
8363,L10Q2PGA80,Temporin-1TGa,Rana tagoi,1383.7 Da,13 aa,9.69,Antimicrobial,FLPIIGKLLSGIL,1 Target: E. coli MIC: 138.37 ug/ml (100 ...,S. aureus,MIC,15.000000,uM
8364,L10Q800R30,PBN1,Phyllomedusa bicolor,2043.5 Da,20 aa,9.69,Antimicrobial,FLSLIPHIVSGVAALAKHLG,1 Target: S. aureus MIC: 25.5438 ug/ml (1...,S. aureus,MIC,25.543800,ug/ml


In [7]:
# 写入新文件
# 新建一个表格，按照原本规定的格式输出样本数据
finaldata=pd.DataFrame(columns=['database_id','name','n_terminal','sequence','c_terminal','other_modification','target','measure','value','unit','database'])
finaldata[['name','sequence','target','measure','value','unit','database_id']]=new[['name','sequence','target','measure','value','unit','id']]
finaldata[['n_terminal','c_terminal','other_modification']]='none'
finaldata['database']='LAMP'
finaldata.to_excel(r'LAMP.xlsx',index=False)
finaldata

,database_id,name,n_terminal,sequence,c_terminal,other_modification,target,measure,value,unit,database
0,L01A000002,D104A_HUMAN,none,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,none,none,Staphylococcus carnosus TM300,MIC,4.500000,ug/ml,LAMP
1,L01A000002,D104A_HUMAN,none,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,none,none,Staphylococcus carnosus TM300,MIC,0.787746,uM,LAMP
2,L01A000002,D104A_HUMAN,none,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,none,none,Escherichia coli BL21,MIC,100.000000,ug/ml,LAMP
3,L01A000002,D104A_HUMAN,none,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,none,none,Escherichia coli BL21,MIC,17.505500,uM,LAMP
4,L01A000002,D104A_HUMAN,none,ELDRICGYGTARCRKKCRSQEYRIGRCPNTYACCLRKWDESLLNRTKP,none,none,Saccharomyces cerevisiae ATCC9763,MIC,100.000000,ug/ml,LAMP
...,...,...,...,...,...,...,...,...,...,...,...
8361,L10Q2PGA80,Temporin-1TGa,none,FLPIIGKLLSGIL,none,none,E. coli,MIC,100.000000,uM,LAMP
8362,L10Q2PGA80,Temporin-1TGa,none,FLPIIGKLLSGIL,none,none,S. aureus,MIC,20.755500,ug/ml,LAMP
8363,L10Q2PGA80,Temporin-1TGa,none,FLPIIGKLLSGIL,none,none,S. aureus,MIC,15.000000,uM,LAMP
8364,L10Q800R30,PBN1,none,FLSLIPHIVSGVAALAKHLG,none,none,S. aureus,MIC,25.543800,ug/ml,LAMP
